In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_auc_score
)

warnings.filterwarnings('ignore')

In [19]:
#########################
# cargamos los datasets #
#########################

X_train = pd.read_csv("../../data/processed/X_train.csv")
X_val = pd.read_csv("../../data/processed/X_val.csv")
X_test = pd.read_csv("../../data/processed/X_test.csv")
y_train = pd.read_csv("../../data/processed/y_train.csv").squeeze()
y_val = pd.read_csv("../../data/processed/y_val.csv").squeeze()
y_test = pd.read_csv("../../data/processed/y_test.csv").squeeze()


In [20]:
######################
# funcion evaluación #
######################

'''Hace no mucho le vi esta función a dotcsv y me encantó para enseñar
los resultados de forma clara, desde entonces me gusta usarla en todos 
mis modelos'''

def mostrar_resultados(modelo, X, y, nombre):
    predicciones = modelo.predict(X)
    probas = modelo.predict_proba(X)[:, 1]
    print(f"\nResultados de {nombre}:")
    print("Accuracy:", accuracy_score(y, predicciones))
    print("Precision:", precision_score(y, predicciones))
    print("Recall:", recall_score(y, predicciones))
    print("F1 Score:", f1_score(y, predicciones))
    print("ROC AUC:", roc_auc_score(y, probas))
    print("Matriz de confusión:\n", confusion_matrix(y, predicciones))


In [21]:
###############################
# stratified cross-validation #
###############################

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [22]:
########################
# Tuning Random Forest #
########################

#Uso de best stimator con el cv de 5 folds

parametros_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']}

busqueda_rf = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_distributions=parametros_rf,
    scoring='roc_auc',
    cv=cv,
    n_iter=20,
    verbose=1,
    random_state=42,
    n_jobs=-1)

busqueda_rf.fit(X_train, y_train)

modelo_rf = busqueda_rf.best_estimator_
print("Mejores hiperparámetros para Random Forest:")
print(busqueda_rf.best_params_)

mostrar_resultados(modelo_rf, X_val, y_val, "Random Forest Optimizado")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Mejores hiperparámetros para Random Forest:
{'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 30}

Resultados de Random Forest Optimizado:
Accuracy: 0.8687230989956959
Precision: 0.7903137039075399
Recall: 0.6147260273972602
F1 Score: 0.6915482783529978
ROC AUC: 0.9253908744550633
Matriz de confusión:
 [[7041  381]
 [ 900 1436]]


In [23]:
########################
#   Tuning LightGBM    #
########################

parametros_lgbm = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [15, 31, 63],
    'max_depth': [-1, 10, 20],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

busqueda_lgbm = RandomizedSearchCV(
    estimator=LGBMClassifier(random_state=42, n_jobs=-1),
    param_distributions=parametros_lgbm,
    scoring='roc_auc',
    cv=cv,
    n_iter=30,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

busqueda_lgbm.fit(X_train, y_train)

modelo_lgbm = busqueda_lgbm.best_estimator_
print("Mejores hiperparámetros para LightGBM:")
print(busqueda_lgbm.best_params_)


mostrar_resultados(modelo_lgbm, X_val, y_val, "LightGBM Optimizado")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Info] Number of positive: 7009, number of negative: 22265
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001463 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 29274, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239427 -> initscore=-1.155821
[LightGBM] [Info] Start training from score -1.155821
Mejores hiperparámetros para LightGBM:
{'subsample': 1.0, 'num_leaves': 31, 'n_estimators': 100, 'min_child_samples': 20, 'max_depth': -1, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

Resultados de LightGBM Optimizado:
Accuracy: 0.8744619799139168
Precision: 0.7778889444722361
Recall: 0.665667808219178
F1 Score: 0.7174163783160323
ROC AUC: 0.9348780975109171
Matriz de confusió

In [26]:
#############################################
# Guardo el mejor modelo que es el LightGBM #
#############################################


os.makedirs("../../data/models", exist_ok=True)
with open("../../data/models/modelo_lgbm.pkl", "wb") as f:
    pickle.dump(modelo_lgbm, f)

with open("../../data/models/modelo_rf.pkl", "wb") as f:
    pickle.dump(modelo_rf, f)

print("Modelos guardados")


Modelos guardados


He investigado cómo ajustar los hiperparámetros de Random Forest y LightGBM usando validación cruzada para evitar overfitting. Usé una búsqueda aleatoria porque es más rápida que un grid completo y permite probar más combinaciones sin que tarde horas.

Para Random Forest ajusté cosas como el número de árboles, la profundidad máxima, y cuántas muestras necesita cada hoja. El modelo final ha subido bastante en métricas como ROC AUC y precision, aunque ha bajado un poco el recall.

En el caso de LightGBM, ajusté parámetros como el learning_rate, el número de hojas y cuánto se puede usar de cada columna y de cada muestra. Este modelo ha dado el mejor resultado global, con una buena combinación de todas las métricas y el ROC AUC más alto.

